In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
place=pd.read_csv("place_data.csv")
place.head(2)

,Unnamed: 0,이름,코드,태그
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장


In [3]:
place.shape

(465, 4)

In [4]:
type(place['태그'])

pandas.core.series.Series

In [5]:
place['tag']=place['태그'].copy()
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장


In [6]:
place['tag'][0]

'#먹어야산다#분식#정대후문#포장#혼밥#가성비'

In [7]:
#tag를 '#'기준으로 split

place['tag']=place['tag'].str.split(pat="#")
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,"[, 먹어야산다, 분식, 정대후문, 포장, 혼밥, 가성비]"
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,"[, 먹어야산다, 이공계, 경양식, 버거, 맥주, 치킨, 가성비, 저렴, 포장]"


In [8]:
place['tag']=place['tag'].apply(lambda x: " ".join(x))
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,먹어야산다 분식 정대후문 포장 혼밥 가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,먹어야산다 이공계 경양식 버거 맥주 치킨 가성비 저렴 포장


In [9]:
place.tag.head(2)

0              먹어야산다 분식 정대후문 포장 혼밥 가성비
1     먹어야산다 이공계 경양식 버거 맥주 치킨 가성비 저렴 포장
Name: tag, dtype: object

In [10]:
count = CountVectorizer()
count_matrix = count.fit_transform(place['tag'])

In [11]:
count_matrix.shape

(465, 374)

In [12]:
#tag 코사인 유사도 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.40824829 0.57735027 ... 0.40824829 0.23570226 0.16666667]
 [0.40824829 1.         0.23570226 ... 0.33333333 0.19245009 0.13608276]
 [0.57735027 0.23570226 1.         ... 0.35355339 0.20412415 0.14433757]
 ...
 [0.40824829 0.33333333 0.35355339 ... 1.         0.57735027 0.40824829]
 [0.23570226 0.19245009 0.20412415 ... 0.57735027 1.         0.23570226]
 [0.16666667 0.13608276 0.14433757 ... 0.40824829 0.23570226 1.        ]]


In [13]:
indices = pd.Series(place.index, index=place.코드).drop_duplicates()
print(indices)

코드
Num=a_nam_0002      0
Num=a_nam_0041      1
Num=a_nam_829       2
Num=a_nam_0146      3
Num=a_nam_0319      4
                 ... 
Num=a_nam_897     460
Num=a_nam_1069    461
Num=a_nam_1104    462
Num=a_nam_1107    463
Num=a_nam_1198    464
Length: 465, dtype: int64


In [14]:
indices["Num=a_nam_897"]

460

In [15]:
def place_REC(코드, cosine_sim=cosine_sim):
    #입력한 코드로 부터 인덱스 가져오기
    idx = indices[코드]

    # 모든 장소에 대해서 해당 장소와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 장소들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 장소를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 장소의 인덱스 받아옴
    place_indices = [i[0] for i in sim_scores]
    
    #기존 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_place = place.iloc[place_indices].copy()
    result_place['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 tag부분만 제거, 코드와 스코어만 보이게 함
    del result_place['tag']

    # 가장 유사한 10개의 장소를 반환
    return result_place

In [20]:
place_REC("Num=a_nam_787")

,Unnamed: 0,이름,코드,태그,score
183,183,비야 이공후문,Num=a_nam_0177,#먹어야산다#한식#이공계,0.57735
414,414,누리수제닭강정,Num=a_nam_990,#먹어야산다#이공계#닭강정,0.57735
95,95,나정순할매쭈꾸미,Num=a_nam_845,#먹어야산다#쭈꾸미#이공계#안암로터리,0.50000
139,139,박가네 뼈다귀 해장국,Num=a_nam_0181,#먹어야산다#한식#이공계#배달,0.50000
204,204,두부촌,Num=a_nam_720,#먹어야산다,0.50000
317,317,스시진,Num=a_nam_0205,#먹어야산다#일식#이공계#배달,0.50000
344,344,구룡포전어횟집,Num=a_nam_840,#먹어야산다#회,0.50000
345,345,산학관 학생식당,Num=a_nam_1054,#먹어야산다,0.50000
371,371,엄자손만두,Num=a_nam_915,#먹어야산다,0.50000
380,380,박쉐프의 스키야끼,Num=a_nam_725,#먹어야산다,0.50000
